# Ejercicios ensembling
En este ejercicio vas a realizar prediciones sobre un dataset de ciudadanos indios diabéticos. Se trata de un problema de clasificación en el que intentaremos predecir 1 (diabético) o 0 (no diabético). Para facilitarnos las cosas, vamos a coger un dataset donde todas las variables son numércias.

## 1. Carga las librerías que consideres comunes al notebook

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection

In [2]:
from sklearn.model_selection import  cross_val_score

## 2. Lee los datos de esta [dirección](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv)
Los nombres de columnas son:
```Python
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
```

In [3]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']

df = pd.read_csv(url, names=names)
df.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


## 3. Bagging
Para este apartado tendrás que crear un ensemble utilizando la técnica de bagging ([BaggingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html)), mediante la cual combinarás 100 [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html). Recuerda utilizar también [cross validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) con 10 kfolds.

**Para este apartado y siguientes, no hace falta que dividas en train/test**, por hacerlo más sencillo. Simplemente divide tus datos en features y target.

Además, para poder reproducir el algoritmo numerosas veces, establece una semilla.

In [4]:
array = df.values
X = array[:, 0:8]
Y = array[:, 8]
seed = 42

In [5]:
kfold = model_selection.KFold(n_splits=10)
dtc = DecisionTreeClassifier()

bagging = BaggingClassifier(base_estimator=dtc,
                         n_estimators=100,
                         random_state=seed)

results_bagg = model_selection.cross_val_score(bagging, X, Y, cv=kfold).mean()
# results_bagg = model_selection.cross_val_score(model, X, Y, cv=10).mean() # Este tb valdria
results_bagg

0.775974025974026

In [6]:
model_selection.cross_val_score(bagging, X, Y, cv=10)

array([0.7012987 , 0.77922078, 0.79220779, 0.66233766, 0.76623377,
       0.79220779, 0.83116883, 0.85714286, 0.72368421, 0.80263158])

## 4. Random Forest
En este caso entrena un [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) con 100 árboles y un `max_features` de 3. También con validación cruzada

In [7]:
model = RandomForestClassifier(n_estimators = 100,
                              max_features = 3,
                              random_state = seed)
results_rf = model_selection.cross_val_score(model, X, Y, cv=10).mean()
results_rf

0.768198906356801

## 5. AdaBoost
Implementa un [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html) con 30 árboles.

In [8]:
model = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth=2),
                           n_estimators = 30,
                          random_state=42)

results_ada = model_selection.cross_val_score(model, X, Y, cv=10).mean()
results_ada

0.7434039644565961

In [9]:
model_selection.cross_val_score(model, X, Y, cv=10)

array([0.75324675, 0.77922078, 0.79220779, 0.64935065, 0.76623377,
       0.72727273, 0.74025974, 0.80519481, 0.69736842, 0.72368421])

## 6. GradientBoosting
Implementa un [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) con 100 estimadores

In [10]:
model = GradientBoostingClassifier(n_estimators = 100,
                                  random_state=seed)

results_gb = model_selection.cross_val_score(model, X, Y, cv=10).mean()
results_gb

0.7604066985645933

## 7. XGBoost
Para este apartado utiliza un [XGBoostClassifier](https://docs.getml.com/latest/api/getml.predictors.XGBoostClassifier.html) con 100 estimadores. XGBoost no forma parte de la suite de modelos de sklearn, por lo que tendrás que instalarlo con pip install

In [11]:
#!pip install xgboost

In [12]:
import seaborn
seaborn.__version__

'0.10.1'

In [40]:
model = XGBClassifier(n_estimators=2000, learning_rate=0.01)

results_xgb = model_selection.cross_val_score(model, X, Y, cv=10).mean()
results_xgb

C:\Users\TheBridge\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:06:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:06:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:06:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:06:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

0.744771018455229

## 8. Resultados
Crea un series con los resultados y sus algoritmos, ordenándolos de mayor a menor

In [33]:
resultados = [results_bagg, results_rf, results_ada, results_gb, results_xgb]
algoritmos = ["Bagging DT", "Random Forest", "Ada Boost", "GradientBoosting", "XGBoost"]

result = pd.Series(resultados, algoritmos).sort_values(ascending=False)
result

Bagging DT          0.775974
Random Forest       0.768199
GradientBoosting    0.760407
Ada Boost           0.743404
XGBoost             0.691302
dtype: float64